In [1]:
import pandas as pd
import numpy as np 
from py2neo import Node,Relationship,Graph, database,NodeMatcher 

CREACION DE LOS NOSOS GEN

In [24]:
graph=Graph()# or Graph('bolt://neo4j@localhost:7687')

In [26]:
genes=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\DDBB\genes.csv")
genes['ENSEMBL'].head
g=genes['ENSEMBL'].unique()
syb=genes['SYMBOL'].unique()
genes2=genes.drop_duplicates(subset=['ENSEMBL']).reset_index()#mirar indices en el siguiente ejecutable(66 y 67)
len(syb)
#genes['ENSEMBL'].head,len(g),len(genes2),genes2.head

25846

In [27]:
genes.iloc[65:69],genes2.iloc[65:69],len(genes2)

(            ENSEMBL    SYMBOL                            GENENAME
 65  ENSG00000004864  SLC25A13  solute carrier family 25 member 13
 66  ENSG00000004866       ST7     suppression of tumorigenicity 7
 67  ENSG00000004866   ST7-OT3        ST7 overlapping transcript 3
 68  ENSG00000004897     CDC27              cell division cycle 27,
     index          ENSEMBL    SYMBOL  \
 65     65  ENSG00000004864  SLC25A13   
 66     66  ENSG00000004866       ST7   
 67     68  ENSG00000004897     CDC27   
 68     69  ENSG00000004939    SLC4A1   
 
                                              GENENAME  
 65                 solute carrier family 25 member 13  
 66                    suppression of tumorigenicity 7  
 67                             cell division cycle 27  
 68  solute carrier family 4 member 1 (Diego blood ...  ,
 35220)

In [22]:
#syb=sorted(syb)
len(syb),syb[0:10]

(25846,
 array(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH',
        'FUCA2', 'GCLC', 'NFYA'], dtype=object))

In [28]:
len(g)

35220

In [9]:
no_en_db=[]
for gen in range(0,len(syb)):
    A=graph.run("MATCH(g:Gene{name:$syb}) return g.name",syb=syb[gen])
    #cuando se corre MERGE se demora ~6 veses más que CREATE (134 s v/s 830)
    if gen!=A:
        no_en_db.append(gen)#lista de simbols que no estan en la listas

In [10]:
len(no_en_db)

25846

In [4]:
genes2.columns

Index(['index', 'ENSEMBL', 'SYMBOL', 'GENENAME'], dtype='object')

In [43]:
for gen in range(0,len(genes2['ENSEMBL'])):
    graph.run("CREATE(:Gene{ENSEMBL:$ensembl,SYB:$syb,name:$name})",ensembl=genes2['ENSEMBL'][gen],syb=genes2['SYMBOL'][gen],name=genes2['GENENAME'][gen])
    #cuando se corre MERGE se demora ~6 veses más (134 s v/s 830)

In [46]:
r=graph.run('MATCH (n:Gene) RETURN count((n));').evaluate()
print('Se crearon',r,'nodos <Gene>')

Se crearon 35220 nodos <Gene>


CREAR RELACIONES DE REGULACION USANDO DOROTHEA

In [2]:
genes=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\DDBB\dorothea_final.csv", delimiter="\t",header=None)
genes2=list(genes.to_records(index=False))#lista de tuplas
len(genes2)

486751

In [3]:
genes.head(), genes2[0:5]

(      0       1
 0  ADNP  ATF7IP
 1  ADNP  DYRK1A
 2  ADNP    TLK1
 3  ADNP   ZMYM4
 4  ADNP   ABCC1,
 [('ADNP', 'ATF7IP'),
  ('ADNP', 'DYRK1A'),
  ('ADNP', 'TLK1'),
  ('ADNP', 'ZMYM4'),
  ('ADNP', 'ABCC1')])

In [5]:
len(genes[0].unique())

1395

In [70]:
genes_dot=pd.concat([genes[0],genes[1]])
genes_dot=genes_dot.unique()
len(genes_dot)

20244

In [57]:
genes2[100:110],len(genes2)

([('ADNP', 'CSF3R'),
  ('ADNP', 'CTA'),
  ('ADNP', 'CTB'),
  ('ADNP', 'CTD'),
  ('ADNP', 'CTRC'),
  ('ADNP', 'CUX1'),
  ('ADNP', 'CUX2'),
  ('ADNP', 'DAPK2'),
  ('ADNP', 'DENND1A'),
  ('ADNP', 'DEPDC5')],
 486751)

CORROBORAR QUE TODOS LOS SIMBOLOS EXISTEN PARA TODAS LAS TUPLAS que se tienen, SI NO EXISTE REVISAR MANUALMENTE ese elemento

In [80]:
genes_dot[0]

'ADNP'

In [81]:
no_en_db=[]
for gen in genes_dot:
    #print("MATCH(a:Gene{name:",tupla[0],") MATCH(b:Gene{name:",tupla[1],") CREATE (a)-[:Regulates]->(b)")
    A=graph.run("MATCH(a:Gene) WHERE a.SYB=$name RETURN a.SYB ",name=gen).evaluate()
    if gen!=A:
        no_en_db.append(gen)
len(no_en_db)

1850

In [84]:
syb_en_dot_no_en_db=list(set(no_en_db))
len(syb_en_dot_no_en_db),syb_en_dot_no_en_db[0:10]

(1850,
 ['GALNTL4',
  'LINC01475',
  'ATP5G2',
  'RPL7AP2',
  'FOXI1',
  'RN7SL47P',
  'CSNK1A1P1',
  'RNF219',
  'HD',
  'LRMP'])

In [40]:
no_en_db1=pd.DataFrame(no_en_db)
no_en_db1.to_csv('no_en_db.csv',index=False)

In [ ]:
relation=[]
for tupla in genes2:
    #print("MATCH(a:Gene{name:",tupla[0],") MATCH(b:Gene{name:",tupla[1],") CREATE (a)-[:Regulates]->(b)")
    A=graph.run("MATCH(a:Gene) WHERE a.SYB=$name RETURN a.SYB ",name=tupla[0]).evaluate()
    B=graph.run("MATCH(b:Gene) WHERE b.SYB=$name1 RETURN b.SYB",name1=tupla[1]).evaluate()
    if tupla[0]!=A or tupla[1]!=B:
        relation.append(tupla)
len(relation)

In [54]:
for tupla in genes2:
    #print("MATCH(a:Gene{name:",tupla[0],") MATCH(b:Gene{name:",tupla[1],") CREATE (a)-[:Regulates]->(b)")
    graph.run("MATCH(a:Gene{SYB:$name}),(b:Gene{SYB:$name1}) CREATE (a)-[:Regulates]->(b)",name=tupla[0],name1=tupla[1])


In [79]:
r=graph.run('MATCH ()-[r:Regulates]->()  return COUNT(r);').evaluate()
print('Se crearon',r,'relaciones <Regulate>')

Se crearon 464635 relaciones <Regulate>


In [ ]:
tupla=genes2[0]
tupla

In [ ]:
relation[0:10]

CREAR RELACIONES DE GDA DISGENET

In [30]:
GDAs=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\DDBB\GDAs_15_9_21.csv")


In [35]:
Desease_list=[] 
Desease_id_list=[]
for i in range(0,len(GDAs)):
#for desease ,id_desease in GDAs_1[["Disease","Disease_id"]]:
    if GDAs["Disease"][i] not in Desease_list:#si no existe el nodo then
        Desease_list.append(GDAs["Disease"][i])
        Desease_id_list.append([GDAs["Disease"][i],GDAs["Disease_id"][i]]) # agregalo

len(Desease_id_list),len(GDAs),GDAs.head()

(9347,
 60478,
    Gene          Gene_Full_Name                                Disease  \
 0  A1BG  alpha-1-B glycoprotein                          Schizophrenia   
 1  NAT2   N-acetyltransferase 2  Malignant neoplasm of urinary bladder   
 2  NAT2   N-acetyltransferase 2                       Bladder Neoplasm   
 3  NAT2   N-acetyltransferase 2           Malignant neoplasm of breast   
 4  NAT2   N-acetyltransferase 2              Coronary Arteriosclerosis   
 
   Disease_id  Score_gda  
 0   C0036341       0.30  
 1   C0005684       0.40  
 2   C0005695       0.40  
 3   C0006142       0.40  
 4   C0010054       0.31  )

In [32]:
for des in Desease_id_list: 
    #print("CREATE(:GENE{name:'"+gen+"'})")
    graph.run("Create(a:Desease{name:$name,CUI:$cui})",name=des[0],cui=des[1])
r=graph.run('MATCH (n:Desease) RETURN count((n));').evaluate()
print('Se crearon',r,'nodos <Desease>')

Se crearon 9347 nodos <Desease>


In [33]:
for i in range(0,len(GDAs)):
    graph.run("MATCH(a:Desease{name:$name}) MATCH(b:Gene{SYB:$name1}) CREATE(a)-[:GDA{GDA_score:$name2}]->(b)",name1=GDAs["Gene"][i],name=GDAs["Disease"][i],name2=GDAs["Score_gda"][i])


In [38]:
r=graph.run('MATCH (:Desease)-[n:GDA]->(:Gene) RETURN count(n);').evaluate()
print('Se crearon',r,' relaciones <GDA>')

Se crearon 57764  relaciones <GDA>


In [50]:
no_encontrado=[]
for i in range(0,len(GDAs)):
    a=graph.run(" MATCH(b:Gene{SYB:$name1}) return b.SYB",name1=GDAs["Gene"][i]).evaluate()
    if a!=GDAs["Gene"][i]:
        no_encontrado.append(GDAs["Gene"][i])

In [89]:
syb_en_dis_no_encontrados=list(set(no_encontrado))
len(syb_en_dis_no_encontrados),syb_en_dis_no_encontrados[0:20]

(573,
 ['MIR204',
  'MIR8061',
  'MIR20B',
  'GHS',
  'FOXI1',
  'MIR325',
  'MIR5588',
  'MIR4492',
  'UGT1A',
  'LINC00475',
  'MIR548B',
  'MIR29A',
  'TTN-AS1',
  'OCA5',
  'RMST',
  'BCL10',
  'MIR22HG',
  'MC4R',
  'MIRLET7E',
  'UOX'])